# <font color=pink>**Extração de dados do FMI usando Python**</font>

In [126]:
# Imports
import requests
import pandas as pd
import json
import numpy as np

Referência: https://datahelp.imf.org/knowledgebase/articles/667681-using-json-restful-web-service

In [127]:
# Request para ver a lista de datasets disponíveis na API no FMI
url = "http://dataservices.imf.org/REST/SDMX_JSON.svc/Dataflow"
response = requests.get(url).json()

# Formatando o json
json_formatado = json.dumps(response, indent=4, sort_keys=True)
print(json_formatado)

{
    "Structure": {
        "@xmlns": "http://www.SDMX.org/resources/SDMXML/schemas/v2_0/message",
        "@xmlns:xsd": "http://www.w3.org/2001/XMLSchema",
        "@xmlns:xsi": "http://www.w3.org/2001/XMLSchema-instance",
        "@xsi:schemaLocation": "http://www.SDMX.org/resources/SDMXML/schemas/v2_0/structure https://registry.sdmx.org/schemas/v2_0/SDMXStructure.xsd http://www.SDMX.org/resources/SDMXML/schemas/v2_0/message https://registry.sdmx.org/schemas/v2_0/SDMXMessage.xsd",
        "Dataflows": {
            "Dataflow": [
                {
                    "@agencyID": "IMF",
                    "@id": "DS-BOP_2017M06",
                    "@isFinal": "true",
                    "@version": "1.0",
                    "@xmlns": "http://www.SDMX.org/resources/SDMXML/schemas/v2_0/structure",
                    "KeyFamilyRef": {
                        "KeyFamilyAgencyID": "IMF",
                        "KeyFamilyID": "BOP_2017M06"
                    },
                    "

In [128]:
# Acessando a chave 'Dataflows' para acessar os dados dentro dela
bases = response['Structure']['Dataflows']['Dataflow']

lista_bases = []

# Iterando sobre a lista de bases de dados
for base in bases:
    dataset_info = {
        'ID': base['@id'],
        'Name': base['Name']['#text']
    }
    lista_bases.append(dataset_info)

# Transformando em df
df = pd.DataFrame(lista_bases)

# Filtra o dataset "Financial Access Survey"
df_filtrado = df[df['Name'].str.contains('Financial Access Survey')]
df_filtrado

,ID,Name
52,DS-FAS_2015,"Financial Access Survey (FAS), 2015"
86,DS-FAS_2017,"Financial Access Survey (FAS), 2017"
131,DS-FAS_2016,"Financial Access Survey (FAS), 2016"
175,DS-FAS_2018,"Financial Access Survey (FAS), 2018"
320,DS-FAS,Financial Access Survey (FAS)


In [129]:
# Endpoint da API para ver estrutura de dados do dataset Financial Access Survey (FAS)
database_id = "FAS"
url = f"http://dataservices.imf.org/REST/SDMX_JSON.svc/DataStructure/{database_id}"

# Request
response = requests.get(url).json()

# Formatando json
json_formatado = json.dumps(response, indent=4, sort_keys=True)
print(json_formatado)

{
    "Structure": {
        "@xmlns": "http://www.SDMX.org/resources/SDMXML/schemas/v2_0/message",
        "@xmlns:xsd": "http://www.w3.org/2001/XMLSchema",
        "@xmlns:xsi": "http://www.w3.org/2001/XMLSchema-instance",
        "@xsi:schemaLocation": "http://www.SDMX.org/resources/SDMXML/schemas/v2_0/structure https://registry.sdmx.org/schemas/v2_0/SDMXStructure.xsd http://www.SDMX.org/resources/SDMXML/schemas/v2_0/message https://registry.sdmx.org/schemas/v2_0/SDMXMessage.xsd",
        "CodeLists": {
            "CodeList": [
                {
                    "@agencyID": "IMF",
                    "@id": "CL_UNIT_MULT",
                    "@isFinal": "true",
                    "@version": "1.0",
                    "@xmlns": "http://www.SDMX.org/resources/SDMXML/schemas/v2_0/structure",
                    "Code": [
                        {
                            "@value": "0",
                            "Description": {
                                "#text": "Uni

In [130]:
# Endpoint da API para acessar os indicadores que vamos usar (referentes às mulheres)
frequency = "A"  # período anual
items_dimension1 = "BR+AE+IN+AR+AO+CV+MZ"  # país Brasil
items_dimension2_F = "FCRAMFHFFA_NUM+FCRODCHFFA_NUM+FCDODCHFFA_NUM+FCAODCHFFA_NUM+FCNAMFHFFA_NUM+FCNODCHFFA_NUM+FCRODCHF_PE_NUM+FCROFNMFHF_NUM+FCDODCHF_PE_NUM"  # indicadores
url = f"http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/{database_id}/{frequency}.{items_dimension1}.{items_dimension2_F}"

# Request
response = requests.get(url).json()

# Formatando json
json_formatado = json.dumps(response, indent=4, sort_keys=True)
print(json_formatado)

{
    "CompactData": {
        "@xmlns": "http://www.SDMX.org/resources/SDMXML/schemas/v2_0/message",
        "@xmlns:xsd": "http://www.w3.org/2001/XMLSchema",
        "@xmlns:xsi": "http://www.w3.org/2001/XMLSchema-instance",
        "@xsi:schemaLocation": "http://www.SDMX.org/resources/SDMXML/schemas/v2_0/message https://registry.sdmx.org/schemas/v2_0/SDMXMessage.xsd http://dataservices.imf.org/compact/FAS http://dataservices.imf.org/compact/FAS.xsd",
        "DataSet": {
            "@xmlns": "http://dataservices.imf.org/compact/FAS",
            "Series": [
                {
                    "@FREQ": "A",
                    "@INDICATOR": "FCDODCHF_PE_NUM",
                    "@REF_AREA": "AO",
                    "@TIME_FORMAT": "P1Y",
                    "@UNIT_MULT": "0",
                    "Obs": [
                        {
                            "@OBS_VALUE": "69980",
                            "@TIME_PERIOD": "2004"
                        },
                      

In [131]:
# Acessando a chave 'Series' para acessar a lista de indicadores
indicadores = response['CompactData']['DataSet']['Series']

# Iterando sobre as listas de indicadores
lista_indicadores = []
for indicador in indicadores:
    for obs in indicador['Obs']:
        dataset_info = {
            'Código do indicador': indicador['@INDICATOR'],
            'Localização': indicador['@REF_AREA'],
            'Período': obs.get('@TIME_PERIOD', np.nan),
            'Valor': obs.get('@OBS_VALUE', np.nan)
        }
        lista_indicadores.append(dataset_info)

# Transformando em um df
df1 = pd.DataFrame(lista_indicadores)
df1

,Código do indicador,Localização,Período,Valor
0,FCDODCHF_PE_NUM,AO,2004,69980
1,FCDODCHF_PE_NUM,AO,2005,95642
2,FCDODCHF_PE_NUM,AO,2006,162834
3,FCDODCHF_PE_NUM,AO,2007,284229
4,FCDODCHF_PE_NUM,AO,2008,355057
5,FCDODCHF_PE_NUM,AO,2009,446519
6,FCDODCHF_PE_NUM,AO,2010,646588
7,FCDODCHF_PE_NUM,AO,2011,1189385
8,FCDODCHF_PE_NUM,AO,2012,1458164
9,FCDODCHF_PE_NUM,AO,2013,1721234


In [132]:
# Endpoint da API para acessar os indicadores que vamos analisar (referentes aos homens)
items_dimension2_M = "FCROFNMFHM_NUM+FCDODCHM_PE_NUM+FCRAMFHMMA_NUM+FCRODCHMMA_NUM+FCDODCHMMA_NUM+FCAODCHMMA_NUM+FCNAMFHMMA_NUM+FCNODCHMMA_NUM+FCRODCHM_PE_NUM"  # indicadores
url = f"http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/{database_id}/{frequency}.{items_dimension1}.{items_dimension2_M}"

# Request
response = requests.get(url).json()

# Acessando a chave 'Series' para acessar a lista de indicadores
indicadores = response['CompactData']['DataSet']['Series']

# Iterando sobre as listas de indicadores
lista_indicadores = []
for indicador in indicadores:
    for obs in indicador['Obs']:
        dataset_info = {
            'Código do indicador': indicador['@INDICATOR'],
            'Localização': indicador['@REF_AREA'],
            'Período': obs.get('@TIME_PERIOD', np.nan),
            'Valor': obs.get('@OBS_VALUE', np.nan)
        }
        lista_indicadores.append(dataset_info)

# Transformando em um df
df2 = pd.DataFrame(lista_indicadores)
df2

,Código do indicador,Localização,Período,Valor
0,FCDODCHM_PE_NUM,AO,2004,195334
1,FCDODCHM_PE_NUM,AO,2005,261491
2,FCDODCHM_PE_NUM,AO,2006,441077
3,FCDODCHM_PE_NUM,AO,2007,716193
4,FCDODCHM_PE_NUM,AO,2008,955641
5,FCDODCHM_PE_NUM,AO,2009,1200665
6,FCDODCHM_PE_NUM,AO,2010,1581830
7,FCDODCHM_PE_NUM,AO,2011,2670751
8,FCDODCHM_PE_NUM,AO,2012,3312166
9,FCDODCHM_PE_NUM,AO,2013,3936412


In [133]:
indicador_nomes = {
    'FCRAMFHFFA_NUM': 'Number of female borrowers of the household sector from all microfinance institutions per 1,000 female adults',
    'FCRAMFHMMA_NUM': 'Number of male borrowers of the household sector from all microfinance institutions per 1,000 male adults',
    'FCRODCHFFA_NUM': 'Number of female borrowers of the household sector from commercial banks per 1,000 female adult',
    'FCRODCHMMA_NUM': 'Number of male borrowers of the household sector from commercial banks per 1,000 male adult',
    'FCDODCHFFA_NUM': 'Number of female depositors of the household sector with commercial banks per 1,000 female adults',
    'FCDODCHMMA_NUM': 'Number of male depositors of the household sector with commercial banks per 1,000 male adults',
    'FCAODCHFFA_NUM': 'Number of women-owned deposit accounts of the household sector with commercial banks per 1,000 female adults',
    'FCAODCHMMA_NUM': 'Number of men-owned deposit accounts of the household sector with commercial banks per 1,000 male adults',
    'FCNAMFHFFA_NUM': 'Number of women-owned loan accounts of the household sector with all microfinance institutions per 1,000 female adults',
    'FCNAMFHMMA_NUM': 'Number of men-owned loan accounts of the household sector with all microfinance institutions per 1,000 male adults',
    'FCNODCHFFA_NUM': 'Number of women-owned loan accounts of the household sector with commercial banks per 1,000 female adults',
    'FCNODCHMMA_NUM': 'Number of men-owned loan accounts of the household sector with commercial banks per 1,000 male adults',
    'FCRODCHF_PE_NUM': 'Number of Borrowers, Commercial banks, of which: household sector borrowers, of which: female borrowers',
    'FCRODCHM_PE_NUM': 'Number of Borrowers, Commercial banks, of which: household sector borrowers, of which: male borrowers',
    'FCROFNMFHF_NUM': 'Non-deposit taking microfinance institutions, of which: household sector borrowers, of which: female borrowers',
    'FCROFNMFHM_NUM': 'Non-deposit taking microfinance institutions, of which: household sector borrowers, of which: male borrowers',
    'FCDODCHF_PE_NUM': 'Commercial banks, of which: household sector depositors, of which: female depositors',
    'FCDODCHM_PE_NUM': 'Commercial banks, of which: household sector depositors, of which: male depositors'
}

# Criando nova coluna com o nome dos indicadores
df1['Nome do Indicador'] = df1['Código do indicador'].map(indicador_nomes)
df2['Nome do Indicador'] = df2['Código do indicador'].map(indicador_nomes)

In [134]:
df1

,Código do indicador,Localização,Período,Valor,Nome do Indicador
0,FCDODCHF_PE_NUM,AO,2004,69980,"Commercial banks, of which: household sector d..."
1,FCDODCHF_PE_NUM,AO,2005,95642,"Commercial banks, of which: household sector d..."
2,FCDODCHF_PE_NUM,AO,2006,162834,"Commercial banks, of which: household sector d..."
3,FCDODCHF_PE_NUM,AO,2007,284229,"Commercial banks, of which: household sector d..."
4,FCDODCHF_PE_NUM,AO,2008,355057,"Commercial banks, of which: household sector d..."
5,FCDODCHF_PE_NUM,AO,2009,446519,"Commercial banks, of which: household sector d..."
6,FCDODCHF_PE_NUM,AO,2010,646588,"Commercial banks, of which: household sector d..."
7,FCDODCHF_PE_NUM,AO,2011,1189385,"Commercial banks, of which: household sector d..."
8,FCDODCHF_PE_NUM,AO,2012,1458164,"Commercial banks, of which: household sector d..."
9,FCDODCHF_PE_NUM,AO,2013,1721234,"Commercial banks, of which: household sector d..."


In [135]:
df2

,Código do indicador,Localização,Período,Valor,Nome do Indicador
0,FCDODCHM_PE_NUM,AO,2004,195334,"Commercial banks, of which: household sector d..."
1,FCDODCHM_PE_NUM,AO,2005,261491,"Commercial banks, of which: household sector d..."
2,FCDODCHM_PE_NUM,AO,2006,441077,"Commercial banks, of which: household sector d..."
3,FCDODCHM_PE_NUM,AO,2007,716193,"Commercial banks, of which: household sector d..."
4,FCDODCHM_PE_NUM,AO,2008,955641,"Commercial banks, of which: household sector d..."
5,FCDODCHM_PE_NUM,AO,2009,1200665,"Commercial banks, of which: household sector d..."
6,FCDODCHM_PE_NUM,AO,2010,1581830,"Commercial banks, of which: household sector d..."
7,FCDODCHM_PE_NUM,AO,2011,2670751,"Commercial banks, of which: household sector d..."
8,FCDODCHM_PE_NUM,AO,2012,3312166,"Commercial banks, of which: household sector d..."
9,FCDODCHM_PE_NUM,AO,2013,3936412,"Commercial banks, of which: household sector d..."


In [139]:
df3 = pd.concat([df1,df2], ignore_index=True)
df3

,Código do indicador,Localização,Período,Valor,Nome do Indicador
0,FCDODCHF_PE_NUM,AO,2004,69980,"Commercial banks, of which: household sector d..."
1,FCDODCHF_PE_NUM,AO,2005,95642,"Commercial banks, of which: household sector d..."
2,FCDODCHF_PE_NUM,AO,2006,162834,"Commercial banks, of which: household sector d..."
3,FCDODCHF_PE_NUM,AO,2007,284229,"Commercial banks, of which: household sector d..."
4,FCDODCHF_PE_NUM,AO,2008,355057,"Commercial banks, of which: household sector d..."
5,FCDODCHF_PE_NUM,AO,2009,446519,"Commercial banks, of which: household sector d..."
6,FCDODCHF_PE_NUM,AO,2010,646588,"Commercial banks, of which: household sector d..."
7,FCDODCHF_PE_NUM,AO,2011,1189385,"Commercial banks, of which: household sector d..."
8,FCDODCHF_PE_NUM,AO,2012,1458164,"Commercial banks, of which: household sector d..."
9,FCDODCHF_PE_NUM,AO,2013,1721234,"Commercial banks, of which: household sector d..."


In [140]:
 # Exportando o arquivo
df3.to_csv('IndicadoresFMI.csv')